### Use template code to load stored NewsAPI data from S3
### Replace variables with your own where is say "key-here"
### Do not post notebooks with keys to GitHub

In [2]:
import os.path
from os import path
import pandas as pd
import json
import boto3 
import os
from io import StringIO

In [3]:
# API Key for News API
secret = 'key here'

In [4]:
# s3 secret - Carlos Keys (don't commit to GitHub)
ACCESS_KEY = "key-here" 
SECRET_KEY = "key-here" 
BUCKET_NAME ='key-here'

In [5]:
# S3 credentials
s3 = boto3.resource(
    's3',
    region_name='us-east-1',
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY
).Bucket(BUCKET_NAME)

json.load_s3 = lambda f: json.load(s3.Object(key=f).get()["Body"])
json.dump_s3 = lambda obj, f: s3.Object(key=f).put(Body=json.dumps(obj))

In [6]:
categories = ['business','entertainment','general','health','science','sports','technology']

## Json files stored
* resources/business_top_headline_data_new.json
* resources/entertainment_top_headline_data_new.json
* resources/general_top_headline_data_new.json
* resources/health_top_headline_data_new.json
* resources/science_top_headline_data_new.json
* resources/sports_top_headline_data_new.json
* resources/technology_top_headline_data_new.json

In [8]:
# Pull data
count =0
for category in categories:
    name = category
    print(name)
    fileName= f'resources/{category}_top_headline_data_new.json'
    print(fileName)
    json_buffer=json.load_s3('Project3/'+fileName)
    category_dataframe = pd.read_json(json_buffer).T
    print(category_dataframe.columns)
    print(category_dataframe.shape)

business
resources/business_top_headline_data_new.json
Index(['articleSentiment', 'articleSummary', 'articleText', 'author',
 'category', 'description', 'publishedAt', 'source', 'title', 'url',
 'urlToImage'],
 dtype='object')
(927, 11)
entertainment
resources/entertainment_top_headline_data_new.json
Index(['articleSentiment', 'articleSummary', 'articleText', 'author',
 'category', 'description', 'publishedAt', 'source', 'title', 'url',
 'urlToImage'],
 dtype='object')
(741, 11)
general
resources/general_top_headline_data_new.json
Index(['articleSentiment', 'articleSummary', 'articleText', 'author',
 'category', 'description', 'publishedAt', 'source', 'title', 'url',
 'urlToImage'],
 dtype='object')
(740, 11)
health
resources/health_top_headline_data_new.json
Index(['articleSentiment', 'articleSummary', 'articleText', 'author',
 'category', 'description', 'publishedAt', 'source', 'title', 'url',
 'urlToImage'],
 dtype='object')
(827, 11)
science
resources/science_top_headline_data_new.json
Index(['articleSentiment', 'articleSummary', 'articleText', 'author',
 'category', 'description', 'publishedAt', 'source', 'title', 'url',
 'urlToImage'],
 dtype='object')
(397, 11)
sports
resources/sports_top_headline_data_new.json
Index(['articleSentiment', 'articleSummary', 'articleText', 'author',
 'category', 'description', 'publishedAt', 'source', 'title', 'url',
 'urlToImage'],
 dtype='object')
(910, 11)
technology
resources/technology_top_headline_data_new.json
Index(['articleSentiment', 'articleSummary', 'articleText', 'author',
 'category', 'description', 'publishedAt', 'source', 'title', 'url',
 'urlToImage'],
 dtype='object')
(734, 11)

In [10]:
# Read Business file
json_buffer=json.load_s3('Project3/resources/business_top_headline_data_new.json')
Business=pd.read_json(json_buffer).T
Business.shape
display(Business)

In [ ]:
# Read Entertainment file
json_buffer1=json.load_s3('Project3/resources/entertainment_top_headline_data_new.json')
Entertainment=pd.read_json(json_buffer1).T
#Entertainment.shape
display(Entertainment)

In [ ]:
# Read General file
json_buffer2=json.load_s3('Project3/resources/general_top_headline_data_new.json')
General=pd.read_json(json_buffer2).T
#General.shape
display(General)

In [ ]:
# Read Health file
json_buffer3=json.load_s3('Project3/resources/health_top_headline_data_new.json')
Health=pd.read_json(json_buffer3).T
#Health.shape
display(Health)

In [ ]:
# Read Science file
json_buffer4=json.load_s3('Project3/resources/science_top_headline_data_new.json')
Science=pd.read_json(json_buffer4).T
#Science.shape
display(Science)

In [ ]:
# Read Sports file
json_buffer5=json.load_s3('Project3/resources/sports_top_headline_data_new.json')
Sports=pd.read_json(json_buffer5).T
#Sports.shape
display(Sports)

In [ ]:
# Read Technology file
json_buffer6=json.load_s3('Project3/resources/technology_top_headline_data_new.json')
Technology=pd.read_json(json_buffer6).T
#Technology.shape
display(Technology)

In [ ]:
# Combine data using pd.concat, confirmed total rows matches sum of tables combined
alldata = pd.concat([Business,Entertainment,General,Health,Science,Sports,Technology], sort=False)
alldata.shape

In [12]:
# Save to Postgres (replace the variables, push to github)

import boto3

secret_name = "key-here"
region_name = "us-east-2"
access_key = "key-here"
secret_key = "key-here"

session = boto3.session.Session(aws_access_key_id=access_key, aws_secret_access_key=secret_key, region_name=region_name)
client = session.client('secretsmanager')
secret_value = client.get_secret_value(SecretId=secret_name)

import json
def get_connection(secret_value):
  return json.loads(secret_value['SecretString'])

connection = get_connection(secret_value)
# Postgres credentials
jdbcHostname = connection['host']
jdbcPort = connection['port']
jdbcDatabase = "postgres"
dialect = "postgresql"
jdbcUsername = connection['username']
jdbcPassword = connection['password']

jdbcUrl = f"jdbc:{dialect}://{jdbcHostname}:{jdbcPort}/{jdbcDatabase}"
connectionProperties = {
  "user" : jdbcUsername,
  "password" : jdbcPassword,
  "driver" : "org.postgresql.Driver" 
}

In [13]:
# Checking jdbcurl is formatted correctly
jdbcUrl

Out[10]: 'jdbc:postgresql://news-sentiment.crziwjajzlrj.us-east-2.rds.amazonaws.com:5432/postgres'

In [14]:
from pyspark.sql import SparkSession
from pyspark import SparkFiles
from pyspark.sql import DataFrame
import pyspark.sql.functions as F

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [15]:
# Create Spark session
app_name = "news_sentiment"
spark = SparkSession.builder.appName(app_name).getOrCreate()

In [16]:
# Check initial columns
category_dataframe.columns

Out[13]: Index(['articleSentiment', 'articleSummary', 'articleText', 'author',
 'category', 'description', 'publishedAt', 'source', 'title', 'url',
 'urlToImage'],
 dtype='object')

In [17]:
# Convert to Spark df
import pandas as pd
pdf = pd.DataFrame(category_dataframe)
df = spark.createDataFrame(pdf)

In [18]:
# View Spark df
display(df)

articleSentiment articleSummary articleText author category description publishedAt source title url urlToImage Photo : Google

Apparently the cat is out of the bag because after images from regulatory filings depicting Google’s next smart speaker leaked earlier this week, Google decided to release an official teaser of the Nest team’s upcoming device. Advertisement

While Google made no mention of the gadget’s official name, price, or release date, just a quick look at the image above basically confirms that the leaked device will indeed be the long-overdue replacement for the original Google Home, which first went on sale way back in 2016. Advertisement

While dropping an official image of what is still technically an unnamed product might seem a bit strange, Google has actually done this before. Photo : Google

Apparently the cat is out of the bag because after images from regulatory filings depicting Google’s next smart speaker leaked earlier this week, Google decided to release an official teaser of the Nest team’s upcoming device.

Advertisement

While Google made no mention of the gadget’s official name, price, or release date, just a quick look at the image above basically confirms that the leaked device will indeed be the long-overdue replacement for the original Google Home, which first went on sale way back in 2016.

Aside from confirming that a new Google speaker is on the way, Google’s teaser (which includes a short supercut posted below) also sheds light on some other aspects of the device. Sporting a rounded rectangular body covered in fabric, the next Google Home’s design nicely bridges the size gap between the Nest Mini and the Google Home Max while still maintaining a cohesive design language across Google’s full range of smart speakers. It’s a much-needed update, as the original Google Home’s design—which sort of looks like a futuristic air diffuser— hasn’t aged particularly well.

As info from the regulatory filings suggested, the next Google Home features a set of indicator lights hidden behind its fabric exterior, while the blue version (which looks to be the same shade currently available on the Nest Mini) indicates that it will probably share some of the same pastel color options used on other Nest products. That said, every year Google tends to add a couple of new hues to its product lineup (blue and coral were the main additions last year), usually during its fall product showcase, so don’t be surprised if some new colors make a debut later this year.

Advertisement

While dropping an official image of what is still technically an unnamed product might seem a bit strange, Google has actually done this before. Last year, following a ton of leaks and rumors regarding the Pixel 4, Google released an official image of the phone in June, which was more than four months before the phone got fully revealed later in October.

So in a way, Google is sort of validating all the tech sleuths constantly searching for hints and leaks for upcoming products, because who knows, if enough info gets uncovered, Google may just release even more details about upcoming products. Until then, we’re just going to have to wait until later this fall when we expect Google to reveal more info about its next smart speaker and everything else the company has in store. Sam Rutherford technology Apparently the cat is out of the bag because after images from regulatory filings depicting Google’s next smart speaker leaked earlier this week, Google decided to release an official teaser of the Nest team’s upcoming device. 2020-07-10T15:01:00Z Map(name -> Gizmodo.com, id -> null) After Images Leak, Google Drops an Official Teaser of Its Next Smart Speaker - Gizmodo https://gizmodo.com/after-images-leak-google-drops-an-official-teaser-of-i-1844335217 https://i.kinja-img.com/gawker-media/image/upload/c_fill,f_auto,fl_progressive,g_center,h_675,pg_1,q_80,w_1200/mtoxirmha5smgmo8svzp.png Angela Lang/CNET

Apple is releasing major new software 

In [19]:
# View Schema
df.printSchema()

root
-- articleSentiment: string (nullable = true)
-- articleSummary: string (nullable = true)
-- articleText: string (nullable = true)
-- author: string (nullable = true)
-- category: string (nullable = true)
-- description: string (nullable = true)
-- publishedAt: string (nullable = true)
-- source: map (nullable = true)
 |-- key: string
 |-- value: string (valueContainsNull = true)
-- title: string (nullable = true)
-- url: string (nullable = true)
-- urlToImage: string (nullable = true)

In [20]:
# Write table to postgres AWS
table = "sentiment_results"
mode = "overwrite" # options are: error, append, overwrite

(df.repartition(1)
 .selectExpr("monotonically_increasing_id()+1 as id", "articleSentiment", "articleSummary", "author",
       "category","publishedAt", "source.name", "title", "url", "urlToImage")
 .write
 .jdbc(jdbcUrl, table, mode, connectionProperties)
)